<a href="https://colab.research.google.com/github/jihyunjeongme/deeplearning-tensflow-3min/blob/master/%5B3%EB%B6%84_%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_9%EC%9E%A5_%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%98_%EB%AF%B8%EB%9E%98_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 딥러닝의 미래 GAN

---

2016년의 가장 뜨거운 감자였던 딥러닝 모델인 GAN(Generative Adversarial Network)은 오토인코더와 같이 결과물을 생성하는 생성 모델중 하나로, 서로 대립하는 두 신경망을 경쟁시켜가며 결과물을 생성 방법을 학습합니다.

# 예시
- 위조지폐범(생성자)과 경찰(구분자)에 대한 이야기
- 고흐 풍 그림으로 다시 그려줌
- 선으로만 그려진 만화를 자동으로 채색
- 모잘이크를 없앤다거나 하는 등의 결과

# 이번 장 목표
------
GAN 모형을 이용하여 MNIST 손글씨 숫자를 무작위로 생성하는 간단한 예제를 만들어 보고, 모델을 조금 확장하여 원하는 숫자에 해당하는 이미지를 생성하는 모델을 학습

# 9.1 GAN 기본 모델 구현하기

In [2]:
# 필요한 라이브러리를 불러들임.

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/datas

In [0]:
# 하이퍼 파마리터들을 설정
# 맨 마지막의 n_noise는 생성자의 입력값으로 사용할 노이즈의 크기
# 랜덤한 노이즈를 입력하고 그 노이즈에서 손글씨 이미지를 무작위로 생성

total_epoch = 100
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [0]:
# 플레이스홀더 설정
# GAN도 비지도 학습, Y를 사용하지 않음.
# 구분자에 넣을 이미지가 실제 이미지와 생성한 가짜 이미지 두 개이고,
# 가짜 이미지는 노이즈에서 생성할 것이므로 노이즈를 입력할 플레이스홀더 Z를 추가하였음.

X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [0]:
# 생성자 신경망에 사용할 변수들을 설정
# 첫 번째 가중치와 편향은 은닉층으로 출력하기 위한 변수들
# 두 번째 가중치와 편향은 출력층에 사용할 변수들
# 따라서 두 번째 가중치의 변수 크기는 실제 이미지의 크기와 같아야 합니다.
# 그 크기는 바로 n_input으로, 28 X 28인 784가 됩니다.

G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))


In [0]:
# 구분자 신경망에 사용할 변수들을 설정
# 은닉층은 생성자와 동일하게 구성
# 구분자는 진짜와 얼마나 가까운가를 판단하는 값으로, 0~1 사이의 값을 출력할 것입니다.
# 따라서 하나의 스칼라값을 출력하도록 구성했습니다.

D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_noise, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))


In [0]:
# 실제 이미지를 판별하는 구분자 신경망과 새성한 이미지를 판별하는 구분자 신경망은 같은 변수를
# 사용해야 함.
# 같은 신경망으로 구분 시켜야 진짜 이미지와 가짜 이미지를 구분하는 특징들을 동시에 잡아낼 수 있기 떄문

In [0]:
# 생성자와 구분자 신경망을 구성

# 먼저 생성자 신경망
def generator(noise_z):
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
  
  return output

In [0]:
# 생성자는 무작위로 생성한 노이즈를 받아 가중치와 편향을 반영하여 은닉층을 만들고, 
# 은닉층에서 실제 이미지와 같은 크기의 결괏값을 출력하는 간단한 구성

In [0]:
# 구분자 신경망
# 0~1 사이의 스칼라값 하나를 출력 하도록 하고, 이를 위한 활성화 함수로 sigmoid 함수 사용
def discriminator(inputs):
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
  
  return output

In [0]:
# 무작위한 노이즈를 만들어주는 간단한 유틸리티 함수를 만들어 둡니다.
def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))

In [28]:
# 노이즈 Z를 이용해 가짜 이미지를 만들 생성자 G를 만들고
# 이 Z가 만든 가짜 이미지와 진짜 이미지 X를 각각 구분자에 넣어 입력한 이미지가 진짜인지
# 판별

G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

ValueError: ignored